In [1]:
from torchvision.transforms.v2 import ToTensor

VERSION = '001'
BATCHSIZE = 32
MAX_EPOCHS = 50
BACKBONE = "resnet50"
MODEL_NAME = "ResNet50_transfer"
NUM_WORKERS = 31

In [2]:

import os
from torchvision.transforms import Compose, Resize, InterpolationMode, ToTensor, Normalize
import torch
from torchvision import transforms
from code.classifier import PneumoniaClassifier, Config



In [3]:

# Load the TensorBoard extension
%reload_ext tensorboard
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
# Enable CUDA Launch Blocking
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# Set float32 matrix multiplication precision to 'medium' to utilize Tensor Cores
torch.set_float32_matmul_precision('medium')
torch.serialization.add_safe_globals([Compose, Resize, set, Config, InterpolationMode, ToTensor, Normalize])

In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Sun Nov 24 19:27:25 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.90                 Driver Version: 565.90         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 Ti   WDDM  |   00000000:02:00.0  On |                  N/A |
|  0%   62C    P2             76W /  285W |   11518MiB /  12282MiB |    100%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

config = Config(
    backbone_name="resnet50",
    transfer_learning=True,
    learning_rate=1e-4,
    batch_size=20,
    max_epochs=100,
    weight_decay=1e-4,
    dropout=0.5,
    num_workers=31,
    model_name="ResNet50_transfer",
    version="001",
    optimizer_name = "sgd"
)

model = PneumoniaClassifier(config, transform)
model.train_model()


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\aszab\miniconda3\envs\pneumonia_detection\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(

  | Name              | Type            | Params | Mode 
--------------------------------------------------------------
0 | accuracy          | BinaryAccuracy  | 0      | train
1 | precision         | BinaryPrecision | 0      | train
2 | recall            | BinaryRecall    | 0      | train
3 | f1                | BinaryF1Score   | 0      | train
4 | feature_extractor | Sequential      | 23.5 M | train
5 | dropout           | Dropout         | 0      | train
6 | classifier        | Linear          | 4.1 K  | train
--------------------------------------------------------

Epoch 0: 100%|██████████| 261/261 [01:20<00:00,  3.22it/s, v_num=0]        
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 261/261 [01:21<00:00,  3.19it/s, v_num=0, val_acc_epoch=0.735, val_precision_epoch=0.749, val_recall_epoch=0.967, val_f1_epoch=0.844]

C:\Users\aszab\miniconda3\envs\pneumonia_detection\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryAccuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
C:\Users\aszab\miniconda3\envs\pneumonia_detection\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryPrecision was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
C:\Users\aszab\miniconda3\envs\pneumonia_detection\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryRecall was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
C:\Users\aszab\miniconda3\env

Epoch 1: 100%|██████████| 261/261 [01:25<00:00,  3.06it/s, v_num=0, val_acc_epoch=0.735, val_precision_epoch=0.749, val_recall_epoch=0.967, val_f1_epoch=0.844, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 261/261 [01:26<00:00,  3.03it/s, v_num=0, val_acc_epoch=0.808, val_precision_epoch=0.803, val_recall_epoch=0.983, val_f1_epoch=0.884, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]

Metric val_loss improved by 0.054 >= min_delta = 0.0. New best score: 0.636


...

Epoch 18: 100%|██████████| 261/261 [01:24<00:00,  3.10it/s, v_num=0, val_acc_epoch=0.920, val_precision_epoch=0.938, val_recall_epoch=0.955, val_f1_epoch=0.947, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 19: 100%|██████████| 261/261 [01:23<00:00,  3.14it/s, v_num=0, val_acc_epoch=0.919, val_precision_epoch=0.936, val_recall_epoch=0.956, val_f1_epoch=0.946, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 20: 100%|██████████| 261/261 [01:23<00:00,  3.12it/s, v_num=0, val_acc_epoch=0.922, val_precision_epoch=0.939, val_recall_epoch=0.957, val_f1_epoch=0.948, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 20: 100%|██████████| 261/261 [01:24<00:00,  3.09it/s, v_num=0, val_acc_ep

Metric val_loss improved by 0.012 >= min_delta = 0.0. New best score: 0.403


Epoch 21: 100%|██████████| 261/261 [01:23<00:00,  3.14it/s, v_num=0, val_acc_epoch=0.923, val_precision_epoch=0.940, val_recall_epoch=0.958, val_f1_epoch=0.949, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 22: 100%|██████████| 261/261 [01:23<00:00,  3.12it/s, v_num=0, val_acc_epoch=0.924, val_precision_epoch=0.941, val_recall_epoch=0.958, val_f1_epoch=0.949, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 23: 100%|██████████| 261/261 [01:23<00:00,  3.14it/s, v_num=0, val_acc_epoch=0.930, val_precision_epoch=0.946, val_recall_epoch=0.961, val_f1_epoch=0.953, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 23: 100%|██████████| 261/261 [01:23<00:00,  3.11it/s, v_num=0, val_acc_ep

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.403. Signaling Trainer to stop.


Epoch 23: 100%|██████████| 261/261 [01:23<00:00,  3.11it/s, v_num=0, val_acc_epoch=0.918, val_precision_epoch=0.937, val_recall_epoch=0.953, val_f1_epoch=0.945, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]


In [7]:
checkpoint = torch.load(f"../checkpoints/{config.model_name}.ckpt", weights_only=True)

# Access the metadata
metadata = checkpoint.get("metadata", {})

In [8]:
# Initialize the models
model = PneumoniaClassifier(config, transform)
# Test the models
metadata = model.test_model(f"../checkpoints/{config.model_name}.ckpt")

# Print the metadata
print(metadata)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 32/32 [00:11<00:00,  2.76it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_acc_epoch         0.4166666567325592
      test_acc_step          0.664646327495575
      test_f1_epoch         0.2540983557701111
      test_f1_step          0.11128944903612137
        test_loss           0.7113064527511597
  test_precision_epoch      0.6326530575752258
   test_precision_step      0.2386043816804886
    test_recall_epoch       0.15897436439990997
    test_recall_step        0.07374874502420425
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
None


In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

config = Config(
    backbone_name="resnet50",
    transfer_learning=False,
    learning_rate=1e-4,
    batch_size=20,
    max_epochs=100,
    weight_decay=1e-4,
    dropout=0.5,
    num_workers=31,
    model_name="ResNet50",
    version="001",
    optimizer_name = "sgd"
)

model = PneumoniaClassifier(config, transform)
model.train_model()

checkpoint = torch.load(f"../checkpoints/{config.model_name}.ckpt", weights_only=True)

# Access the metadata
metadata = checkpoint.get("metadata", {})

# Initialize the models
model = PneumoniaClassifier(config, transform)
# Test the models
metadata = model.test_model(f"../checkpoints/{config.model_name}.ckpt")

# Print the metadata
print(metadata)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\aszab\miniconda3\envs\pneumonia_detection\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Users\aszab\Documents\Studium\Modules\ComputerScienceProject\MedicalImaging\pneumonia_detection\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type            | Params | Mode 
--------------------------------------------------------------
0 | accuracy          | BinaryAccuracy  | 0      | train
1 | precision         | BinaryPrecision | 0      | train
2 | recall            | BinaryRecall    | 0      | train
3 | f1                | BinaryF1Score   | 0      | train
4 | feature_extractor | Sequential      | 23.5 M | train
5 | dropout           | Dropout         | 0      | train
6 | classifier        | Linear          | 4.1 K  | train

Epoch 0: 100%|██████████| 261/261 [01:30<00:00,  2.90it/s, v_num=0]        
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 261/261 [01:30<00:00,  2.88it/s, v_num=0, val_acc_epoch=0.733, val_precision_epoch=0.749, val_recall_epoch=0.964, val_f1_epoch=0.843]

Metric val_loss improved. New best score: 0.751


Epoch 1: 100%|██████████| 261/261 [01:30<00:00,  2.87it/s, v_num=0, val_acc_epoch=0.733, val_precision_epoch=0.749, val_recall_epoch=0.964, val_f1_epoch=0.843, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 261/261 [01:31<00:00,  2.85it/s, v_num=0, val_acc_epoch=0.806, val_precision_epoch=0.801, val_recall_epoch=0.983, val_f1_epoch=0.882, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]

Metric val_loss improved by 0.089 >= min_delta = 0.0. New best score: 0.662


...

Epoch 40: 100%|██████████| 261/261 [01:30<00:00,  2.90it/s, v_num=0, val_acc_epoch=0.932, val_precision_epoch=0.947, val_recall_epoch=0.962, val_f1_epoch=0.954, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 41: 100%|██████████| 261/261 [01:30<00:00,  2.89it/s, v_num=0, val_acc_epoch=0.932, val_precision_epoch=0.949, val_recall_epoch=0.960, val_f1_epoch=0.954, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 42: 100%|██████████| 261/261 [01:29<00:00,  2.92it/s, v_num=0, val_acc_epoch=0.933, val_precision_epoch=0.948, val_recall_epoch=0.962, val_f1_epoch=0.955, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 42: 100%|██████████| 261/261 [01:30<00:00,  2.89it/s, v_num=0, val_acc_ep

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.384. Signaling Trainer to stop.


Epoch 42: 100%|██████████| 261/261 [01:30<00:00,  2.89it/s, v_num=0, val_acc_epoch=0.934, val_precision_epoch=0.950, val_recall_epoch=0.962, val_f1_epoch=0.956, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 32/32 [00:12<00:00,  2.50it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_acc_epoch         0.37339743971824646
      test_acc_step         0.7227854132652283
      test_f1_epoch        0.015113350003957748
      test_f1_step         0.013151109218597412
        test_loss           0.7556480765342712
  test_precision_epoch      0.4285714328289032
   test_precision_step      0.20604389905929565
    test_recall_epoch      0.007692307699471712
    test_recall_step       0.006856540217995644
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
None


In [10]:
transform = transforms.Compose([
    transforms.Resize((1000, 1000)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

config = Config(
    backbone_name="resnet50",
    transfer_learning=True,
    learning_rate=1e-4,
    batch_size=20,
    max_epochs=100,
    weight_decay=1e-4,
    dropout=0.5,
    num_workers=31,
    model_name="ResNet50_transfer_1k",
    version="001",
    optimizer_name = "sgd"
)

model = PneumoniaClassifier(config, transform)
model.train_model()

checkpoint = torch.load(f"../checkpoints/{config.model_name}.ckpt", weights_only=True)

# Access the metadata
metadata = checkpoint.get("metadata", {})

# Initialize the models
model = PneumoniaClassifier(config, transform)
# Test the models
metadata = model.test_model(f"../checkpoints/{config.model_name}.ckpt")

# Print the metadata
print(metadata)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type            | Params | Mode 
--------------------------------------------------------------
0 | accuracy          | BinaryAccuracy  | 0      | train
1 | precision         | BinaryPrecision | 0      | train
2 | recall            | BinaryRecall    | 0      | train
3 | f1                | BinaryF1Score   | 0      | train
4 | feature_extractor | Sequential      | 23.5 M | train
5 | dropout           | Dropout         | 0      | train
6 | classifier        | Linear          | 4.1 K  | train
--------------------------------------------------------------
4.1 K     Trainable params
23.5 M    Non-trainable params
23.5 M    Total params
94.049    Total estimated model params size (MB)
156       Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 261/261 [17:39<00:00,  0.25it/s, v_num=0]        
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 261/261 [17:44<00:00,  0.25it/s, v_num=0, val_acc_epoch=0.739, val_precision_epoch=0.743, val_recall_epoch=0.992, val_f1_epoch=0.850]

Metric val_loss improved. New best score: 0.772


Epoch 1: 100%|██████████| 261/261 [17:38<00:00,  0.25it/s, v_num=0, val_acc_epoch=0.739, val_precision_epoch=0.743, val_recall_epoch=0.992, val_f1_epoch=0.850, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 261/261 [17:33<00:00,  0.25it/s, v_num=0, val_acc_epoch=0.742, val_precision_epoch=0.742, val_recall_epoch=1.000, val_f1_epoch=0.852, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 261/261 [17:37<00:00,  0.25it/s, v_num=0, val_acc_epoch=0.742, val_precision_epoch=0.742, val_recall_epoch=1.000, val_f1_epoch=0.852, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]

Metric val_loss improved by 0.017 >= min_delta = 0.0. New best score: 0.755


...

Epoch 21: 100%|██████████| 261/261 [15:59<00:00,  0.27it/s, v_num=0, val_acc_epoch=0.910, val_precision_epoch=0.909, val_recall_epoch=0.976, val_f1_epoch=0.941, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 22: 100%|██████████| 261/261 [15:29<00:00,  0.28it/s, v_num=0, val_acc_epoch=0.905, val_precision_epoch=0.907, val_recall_epoch=0.972, val_f1_epoch=0.938, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 23: 100%|██████████| 261/261 [15:22<00:00,  0.28it/s, v_num=0, val_acc_epoch=0.903, val_precision_epoch=0.904, val_recall_epoch=0.972, val_f1_epoch=0.937, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 23: 100%|██████████| 261/261 [15:24<00:00,  0.28it/s, v_num=0, val_acc_ep

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.652. Signaling Trainer to stop.


Epoch 23: 100%|██████████| 261/261 [15:24<00:00,  0.28it/s, v_num=0, val_acc_epoch=0.911, val_precision_epoch=0.917, val_recall_epoch=0.968, val_f1_epoch=0.942, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 32/32 [01:05<00:00,  0.49it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_acc_epoch                0.375
      test_acc_step         0.7327309250831604
      test_f1_epoch                 0.0
      test_f1_step                  0.0
        test_loss           0.7160292267799377
  test_precision_epoch              0.0
   test_precision_step              0.0
    test_recall_epoch               0.0
    test_recall_step                0.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
None


In [11]:
transform = transforms.Compose([
    transforms.Resize((1000, 1000)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

config = Config(
    backbone_name="resnet50",
    transfer_learning=False,
    learning_rate=1e-4,
    batch_size=20,
    max_epochs=100,
    weight_decay=1e-4,
    dropout=0.5,
    num_workers=31,
    model_name="ResNet50_1k",
    version="001",
    optimizer_name = "sgd"
)

model = PneumoniaClassifier(config, transform)
model.train_model()

checkpoint = torch.load(f"../checkpoints/{config.model_name}.ckpt", weights_only=True)

# Access the metadata
metadata = checkpoint.get("metadata", {})

# Initialize the models
model = PneumoniaClassifier(config, transform)
# Test the models
metadata = model.test_model(f"../checkpoints/{config.model_name}.ckpt")

# Print the metadata
print(metadata)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type            | Params | Mode 
--------------------------------------------------------------
0 | accuracy          | BinaryAccuracy  | 0      | train
1 | precision         | BinaryPrecision | 0      | train
2 | recall            | BinaryRecall    | 0      | train
3 | f1                | BinaryF1Score   | 0      | train
4 | feature_extractor | Sequential      | 23.5 M | train
5 | dropout           | Dropout         | 0      | train
6 | classifier        | Linear          | 4.1 K  | train
--------------------------------------------------------------
23.5 M    Trainable params
0         Non-trainable params
23.5 M    Total params
94.049    Total estimated model params size (MB)
156       Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 261/261 [13:31<00:00,  0.32it/s, v_num=0]        
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 261/261 [13:37<00:00,  0.32it/s, v_num=0, val_acc_epoch=0.737, val_precision_epoch=0.749, val_recall_epoch=0.970, val_f1_epoch=0.846]

Metric val_loss improved. New best score: 0.759


Epoch 1: 100%|██████████| 261/261 [15:11<00:00,  0.29it/s, v_num=0, val_acc_epoch=0.737, val_precision_epoch=0.749, val_recall_epoch=0.970, val_f1_epoch=0.846, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 261/261 [15:04<00:00,  0.29it/s, v_num=0, val_acc_epoch=0.742, val_precision_epoch=0.742, val_recall_epoch=1.000, val_f1_epoch=0.852, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 261/261 [15:06<00:00,  0.29it/s, v_num=0, val_acc_epoch=0.742, val_precision_epoch=0.742, val_recall_epoch=1.000, val_f1_epoch=0.852, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]

Metric val_loss improved by 0.012 >= min_delta = 0.0. New best score: 0.747


...

Epoch 15: 100%|██████████| 261/261 [15:54<00:00,  0.27it/s, v_num=0, val_acc_epoch=0.885, val_precision_epoch=0.876, val_recall_epoch=0.984, val_f1_epoch=0.927, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 15: 100%|██████████| 261/261 [16:01<00:00,  0.27it/s, v_num=0, val_acc_epoch=0.881, val_precision_epoch=0.874, val_recall_epoch=0.981, val_f1_epoch=0.924, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]

Metric val_loss improved by 0.005 >= min_delta = 0.0. New best score: 0.659


Epoch 16:  44%|████▍     | 115/261 [08:00<10:09,  0.24it/s, v_num=0, val_acc_epoch=0.881, val_precision_epoch=0.874, val_recall_epoch=0.981, val_f1_epoch=0.924, train_acc_epoch=0.000, train_precision_epoch=0.000, train_recall_epoch=0.000, train_f1_epoch=0.000]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
transform = transforms.Compose([
    transforms.Resize((1000, 1000)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

config = Config(
    backbone_name="resnet18",
    transfer_learning=True,
    learning_rate=1e-4,
    batch_size=20,
    max_epochs=100,
    weight_decay=1e-4,
    dropout=0.5,
    num_workers=31,
    model_name="ResNet18_transfer",
    version="001",
    optimizer_name = "sgd"
)

model = PneumoniaClassifier(config, transform)
model.train_model()

checkpoint = torch.load(f"../checkpoints/{config.model_name}.ckpt", weights_only=True)

# Access the metadata
metadata = checkpoint.get("metadata", {})

# Initialize the models
model = PneumoniaClassifier(config, transform)
# Test the models
metadata = model.test_model(f"../checkpoints/{config.model_name}.ckpt")

# Print the metadata
print(metadata)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((1000, 1000)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

config = Config(
    backbone_name="resnet18",
    transfer_learning=False,
    learning_rate=1e-4,
    batch_size=20,
    max_epochs=100,
    weight_decay=1e-4,
    dropout=0.5,
    num_workers=31,
    model_name="ResNet18",
    version="001",
    optimizer_name = "sgd"
)

model = PneumoniaClassifier(config, transform)
model.train_model()

checkpoint = torch.load(f"../checkpoints/{config.model_name}.ckpt", weights_only=True)

# Access the metadata
metadata = checkpoint.get("metadata", {})

# Initialize the models
model = PneumoniaClassifier(config, transform)
# Test the models
metadata = model.test_model(f"../checkpoints/{config.model_name}.ckpt")

# Print the metadata
print(metadata)

In [19]:
# Start TensorBoard and point it to the log directory
%tensorboard --logdir tb_logs/ResNet50_transfer/version_2

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 39112), started 2 days, 4:06:44 ago. (Use '!kill 39112' to kill it.)

# Add Regularization